In [153]:
from numpy import *
import json
import datetime
import numpy as np  

linlist = []

In [154]:
#讀檔2 目前沒用到
def readDataFile(file_name):
    dataFile = open(file_name)          #開檔
    lines = dataFile.read().split('\n') #每行分開，存到lines， lines下面會用到
    dataFile.close()                    #讀完關掉
    
    for i in range(len(lines)):             #lines有幾行就做幾次
        record = lines[i].split(' ')        #用' '分開空格
        print record[:-1]                   #print出來看看
        for j in range(len(record)-1):      #最後會有一個莫名其妙的空值 故用len(record)-1修正矩陣長度-1
            coord[i][j]=coord[i][record[j]] #匯入coord 我們最後回傳的矩陣就是coord
    return coord                            #回傳coord

In [155]:
#讀檔
def readDataFile(file_name,linlist):
    dataFile = open(file_name)          #開檔
    lines = dataFile.read().split('\n') #每行分開，存到lines， lines下面會用到
    dataFile.close()                    #讀完關掉
    
    for i in range(len(lines)-1):                   #lines有幾行就做幾次
        linlist.append(map(int, lines[i].split()))  #用map會直接轉成int格式! 並且分開每個數值
    return linlist                                  #回傳linlist 

In [156]:
gj=readDataFile('input.txt',linlist)     #將此二維陣列存到gj 等等直接呼叫

In [158]:
def createC1(dataSet):
    C1 = [] #C1是大小為1的所有候選集的集合
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])             
    C1.sort()
    return map(frozenset, C1)#use frozen set so we can use it as a key in a dict    

In [159]:
#D表示DataSet，CK表示候選集，minSupport表示最小的支持度，自己設定
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not ssCnt.has_key(can): ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = [] #儲存满足最小支持度要求的項集
    supportData = {} #每個項集的支持度字典
    for key in ssCnt:  #計算所有項集的支持度
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [160]:
#LK是頻繁項集列表，K表示接下來合併的項集中的單個想的個數 {1,2,3} 表示k=3
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2] #前k-2個項次相同時，將兩個集合合併
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList 

In [161]:
#只需要输入DataSet和minSupport即可
def apriori(dataSet, minSupport = 0.3):
    C1 = createC1(dataSet)
    D = map(set, dataSet)
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData#返回頻繁項集和每個項集的minSupport

In [162]:
#輸入的參數分別為：頻繁項、support data dict、自定義的minSupport，返回的是Confidence Rule列表
def generateRules(L, supportData, minConf=0.1):  #支持度是通過scanD得到的字典
    bigRuleList = []
    for i in range(1, len(L)):#只去頻繁項集中元素個数大於2的子集，如{1,2}{1,2,3}，不取{2}{3},etc...
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

In [163]:
#計算可信度，找到滿足minConfidence的要求規則
def calcConf(freqSet, H, supportData, brl, minConf=0.2):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print freqSet-conseq,'-->',conseq,'conf:',conf
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [164]:
#從最初的項集中生成更多的規則
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.2):   #預設minConf=0.2 可由使用者呼叫時直接調整
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #進一步合併項集
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [166]:
rule

[(frozenset({78}), frozenset({39}), 0.7446808510638298),
 (frozenset({3981}), frozenset({39}), 0.810810810810811),
 (frozenset({2958}), frozenset({48}), 0.8043478260869564),
 (frozenset({334}), frozenset({39}), 0.7916666666666667),
 (frozenset({110}), frozenset({38}), 0.9685863874345548),
 (frozenset({281}), frozenset({38}), 0.9677419354838711),
 (frozenset({37}), frozenset({38}), 1.0),
 (frozenset({255}), frozenset({48}), 0.7592592592592593),
 (frozenset({105}), frozenset({38}), 0.9268292682926829),
 (frozenset({1146}), frozenset({39}), 0.7076923076923077),
 (frozenset({49}), frozenset({48}), 0.7681159420289855),
 (frozenset({79}), frozenset({39}), 0.7096774193548387),
 (frozenset({170}), frozenset({38}), 0.9791666666666666),
 (frozenset({47}), frozenset({38}), 0.9705882352941176),
 (frozenset({36}), frozenset({38}), 0.9427083333333334),
 (frozenset({840}), frozenset({38}), 0.96875),
 (frozenset({55}), frozenset({38}), 0.868421052631579),
 (frozenset({175}), frozenset({39}), 0.7045454

In [167]:
#正式的
start = datetime.datetime.now() #紀錄時間
 
L,supportData = apriori(gj,0.1)           #執行apriori  minimum support =0.1   輸出存到 L supportData
rule = generateRules(L,supportData,0.2)   #generateRules  一樣 存到rule

end = datetime.datetime.now()   #程式運行結束時間
runtime = end - start           #計算runtime=end-start

frozenset([39]) --> frozenset([41]) conf: 0.225239269857
frozenset([41]) --> frozenset([39]) conf: 0.763733690197
frozenset([39]) --> frozenset([48]) conf: 0.575076467686
frozenset([48]) --> frozenset([39]) conf: 0.691634033464
frozenset([39]) --> frozenset([38]) conf: 0.204144055254
frozenset([38]) --> frozenset([39]) conf: 0.663311105412
frozenset([41]) --> frozenset([48]) conf: 0.603412512546
frozenset([48]) --> frozenset([41]) conf: 0.214026343895


In [168]:
runtime

datetime.timedelta(0, 168, 196000)

In [169]:
def store(data,path):
    with open(path,'w') as f:
        f.write(data)

In [170]:
freItem_list = []
for tmp in L:
    if len(tmp) !=0:
        for item in tmp:
            freItem_list.append(list(item))
freItem_json = json.dumps(freItem_list,indent=4)
store(freItem_json,'/Users/user/Desktop/freItem.json')

In [171]:
freItem_list

[[41], [39], [32], [38], [48], [41, 39], [48, 39], [38, 39], [48, 41]]

In [172]:
supportData_list = []
for key, value in supportData.iteritems():
    s = str(list(key))+':'+str(value)
    supportData_list.append(s)
supportData_list.sort()
supportData_json = json.dumps(supportData_list,indent=4)
store(supportData_json,'/Users/user/Desktop/supportData.json')

In [173]:
strongRule = []
for tmp in rule:
    s = str(list(tmp[0])[0])+'-->'+str(list(tmp[1])[0])+'---'+str(tmp[2])
    strongRule.append(s)
strongRule_json = json.dumps(strongRule,indent=4)
store(strongRule_json,'/Users/user/Desktop/strongRule.json')

In [174]:
strongRule

['39-->41---0.225239269857',
 '41-->39---0.763733690197',
 '39-->48---0.575076467686',
 '48-->39---0.691634033464',
 '39-->38---0.204144055254',
 '38-->39---0.663311105412',
 '41-->48---0.603412512546',
 '48-->41---0.214026343895']